In [1]:
!ls data/changlin_20171108_2/

README          test4stack.csv  train4stack.csv


In [2]:
import os
import pandas as pd
import matplotlib.pylab as plt

In [3]:
train = pd.read_csv('/Users/guoli/Desktop/kaggle/Porto/train.csv',usecols=['target'])
test = pd.DataFrame()

In [4]:
foldernames = os.popen('ls data').readlines()

In [5]:
for foldername in foldernames:
    foldername = foldername[:-1]
    train[foldername] = pd.read_csv('data/{0}/train4stack.csv'.format(foldername))
    test[foldername] = pd.read_csv('data/{0}/test4stack.csv'.format(foldername))

In [6]:
train.head()

,target,changlin_20171108_1,changlin_20171108_2,guoli_20171108,guoli_20171109,mubi_20171109,tian_20171109
0,0,0.044521,0.046646,0.994861,0.883679,0.047776,0.265382
1,0,0.035425,0.033705,1.047057,0.618817,0.035362,0.256079
2,0,0.017495,0.021387,1.272133,0.641586,0.026082,0.252894
3,0,0.033988,0.014646,0.477906,0.542949,0.023179,0.251246
4,0,0.031818,0.031650,0.769130,0.565832,0.031547,0.248128


In [7]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
 
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_coefficient(preds,dtrain):
    y = dtrain.get_label()
    return 'gini', -gini_normalized(y,preds)

In [8]:
from sklearn.model_selection import KFold

class CustomGridCV(object):
    def __init__(self, X, y, model, metric, griddata, cv=5):
        self.X = X
        self.y = y
        self.model = model
        self.metric = metric
        self.params = self.gridpoints(griddata)
        self.cv = cv
        self.bestScore = None
        self.bestParams = None
        
    def gridpoints(self, data):
        newparams = [{}]
        for k in data.keys():
            params = newparams
            newparams = []
            for v in data[k]:
                for param in params:
                    item = param.copy()
                    item[k]=v
                    newparams.append(item)           
        return newparams
    
    def GridSearch(self):
        for param in self.params:
            self.model.set_params(**param)
            score = self.KFoldScore()
            if self.bestScore==None or self.bestScore<score:
                self.bestScore = score
                self.bestParams = param
            print("Score: {0:.5f}, Params: {1}".format(score,param))
    
    def KFoldScore(self):
        kf = KFold(n_splits=5, shuffle=True, random_state=2)
        y_pred = np.zeros(len(self.y))

        for train_index, test_index in kf.split(self.X):
            train_X, test_X = self.X[train_index], self.X[test_index]
            train_y, test_y = self.y[train_index], self.y[test_index]
            self.model.fit(train_X,train_y)
            #y_pred[test_index] = self.model.predict_proba(test_X)[:,1]
            y_pred[test_index] = self.model.predict(test_X)
            #print y_pred[test_index]

        return self.metric(self.y,y_pred)
    
    def Best(self):
        return self.bestScore, self.bestParams

In [9]:
import pandas as pd
import numpy as np

def PrepareData(df):
    #df = pd.read_csv(trainpath,nrows=nrows)

    features = list(df.columns)
    target = 'target'
    features.remove(target)

    X = np.array(df[features])
    y = np.array(df[target])
    return X, y

In [10]:
from sklearn.linear_model import Lasso, LinearRegression, Ridge

In [11]:
X, y = PrepareData(train)
model = Lasso(positive=True,fit_intercept=False)
griddata = {'alpha':[1e-4,1e-5,1e-6,1e-7,1e-8]}
GCV = CustomGridCV(X, y, model, gini_normalized, griddata)
GCV.GridSearch()

Score: 0.27824, Params: {'alpha': 0.0001}
Score: 0.28382, Params: {'alpha': 1e-05}
Score: 0.28379, Params: {'alpha': 1e-06}
Score: 0.28378, Params: {'alpha': 1e-07}
Score: 0.28378, Params: {'alpha': 1e-08}


In [15]:
model = Lasso(positive=True,fit_intercept=False,alpha=1e-5,)

model.fit(X,y)

print model.coef_

print gini_normalized(y,model.predict(X))

[ 0.          0.79163145  0.00089783  0.          0.1889944   0.        ]
0.283952639086


In [18]:
sorted(zip(list(test.columns),model.coef_),key=lambda x: x[-1])

[('changlin_20171108_1', 0.0),
 ('guoli_20171109', 0.0),
 ('tian_20171109', 0.0),
 ('guoli_20171108', 0.00089783360502210469),
 ('mubi_20171109', 0.1889944031723815),
 ('changlin_20171108_2', 0.79163145294285553)]

In [17]:
y_pred = model.predict(X)

In [20]:
sum(y_pred>1)

0

In [21]:
sum(y_pred<0)

0

In [143]:
#model.predict_proba(np.array(test))[:,1]

In [86]:
result = pd.read_csv('/Users/guoli/Desktop/kaggle/Porto/test.csv',usecols=['id'])
result['target'] = model.predict(np.array(test))

In [87]:
result.head()

,id,target
0,0,0.026815
1,1,0.026101
2,2,0.026099
3,3,0.014812
4,4,0.035259


In [91]:
result.to_csv('5fea_lr_l1.csv',index=False)

In [13]:
X, y = PrepareData(train)
model = Ridge(fit_intercept=False)
griddata = {'alpha':[1e-4,1e-3,1e-2,1e-1,1.]}
GCV = CustomGridCV(X, y, model, gini_normalized, griddata)
GCV.GridSearch()

Score: 0.28323, Params: {'alpha': 0.0001}
Score: 0.28323, Params: {'alpha': 0.001}
Score: 0.28323, Params: {'alpha': 0.01}
Score: 0.28322, Params: {'alpha': 0.1}
Score: 0.28319, Params: {'alpha': 1.0}


In [14]:
model = Ridge(fit_intercept=False,alpha=0.1)

model.fit(X,y)

print model.coef_

print gini_normalized(y,model.predict(X))

[-0.04997164  0.81699995  0.00137139 -0.00102183  0.25743542 -0.00497164]
0.283567612025


In [92]:
! head xgb_nn_encoder.csv

id,target
0,0.0261317432156
1,0.0259512759203
2,0.0256943898781
3,0.021864325779
4,0.0296683108862
5,0.0333643762681
6,0.0218926572291
8,0.0292410953604
10,0.0391363268473


In [93]:
! head xgb_nn_encoder_l1.csv

id,target
0,0.027213944809
1,0.0268504878018
2,0.0268239266324
3,0.0228817182222
4,0.0308930172235
5,0.0347603909417
6,0.0228852173097
8,0.0301340889855
10,0.0407172460428


In [94]:
! head xgb_nn_encoder_lr_l1.csv

id,target
0,0.0268145562496
1,0.0261013701657
2,0.0260991637048
3,0.0148118012637
4,0.0352590130573
5,0.0404777951743
6,0.0172667533561
8,0.0367728049591
10,0.0519336535876
